In [ ]:
import pymc as pm
import pandas as pd
import arviz as az

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import seaborn as sns

def plot_hist(sample, **options):
    """Plot a histogram of goals.
    
    sample: sequence of values
    """
    sns.histplot(sample, stat='probability', discrete=True,
                 alpha=0.5, **options)

def plot_kde(sample, **options):
    """Plot a distribution using KDE.
    
    sample: sequence of values
    """
    sns.kdeplot(sample, cut=0, **options)
    
def legend(**options):
    """Make a legend only if there are labels."""
    handles, labels = plt.gca().get_legend_handles_labels()
    if len(labels):
        plt.legend(**options)

def decorate_heads(ylabel='Probability'):
    """Decorate the axes."""
    plt.xlabel('Number of deaths (k)')
    plt.ylabel(ylabel)
    plt.title('Distribution of deaths')
    legend()

In [ ]:
df = pd.read_csv('Data/apache_sofa_treatment_63.csv', index_col=0)

In [ ]:
df.columns

In [ ]:
df['30-day'].hist()

In [ ]:
n = df.shape[0]
p = len(df[df['3-year']==1])/df.shape[0]
p

In [ ]:
sample_binomial = pm.Binomial.dist(n, p)
sample_binomial = pm.draw(sample_binomial, 1000)

In [ ]:
plot_hist(sample_binomial, label='Binomial')
decorate_heads()

In [ ]:
α = 2
β = 2

In [ ]:
sample_beta = pm.Beta.dist(alpha=α, beta=β)
sample_beta = pm.draw(sample_beta, 1000)
sample_beta.mean()


In [ ]:
def decorate_proportion(ylabel='Likelihood'):
    """Decorate the axes."""
    plt.xlabel('Proportion of heads (x)')
    plt.ylabel(ylabel)
    plt.title('Distribution of proportion')
    legend()

In [ ]:
plot_kde(sample_beta, label='beta')
decorate_proportion()

In [ ]:
α = 2
β = 2
n = df.shape[0]

In [ ]:
with pm.Model() as model1:
    x = pm.Beta('x', alpha=α, beta=α)
    k = pm.Binomial('k', n=n, p=x)
    idata1 = pm.sample_prior_predictive(1000)

In [ ]:
pm.model_to_graphviz(model1)

In [ ]:
sample_prior = idata1.prior['x']
sample_prior.values.mean()

In [ ]:
plot_kde(sample_prior[0])
decorate_proportion()

In [ ]:
idata1

In [ ]:
idata1.prior.k.values[0]

In [ ]:
sample_prior_pred = idata1.prior.k.values[0]
sample_prior_pred.mean()

In [ ]:
plot_hist(sample_prior_pred, label='prior pred')
decorate_heads()

In [ ]:
k_obs = 50

with pm.Model() as model2:
    x = pm.Beta('x', alpha=2, beta=2)
    k = pm.Binomial('k', n=n, p=x, observed=k_obs)
    idata2 = pm.sample(500)

In [ ]:
len(idata2.posterior.x.values[0])

In [ ]:
with model2:
    az.plot_trace(idata2)

In [ ]:
with model2:
    az.plot_posterior(idata2)